In [1]:
from __future__ import annotations

import os
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import yfinance as yf
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import logging

# Load environment variables from .env
try:
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    pass

logging.basicConfig(level=logging.INFO, format="%(asctime)s | %(levelname)s | %(message)s")

# ═══════════════════════════════════════════════════════════════════════════════
# POLYGON CONFIGURATION
# ═══════════════════════════════════════════════════════════════════════════════
POLYGON_API_KEY = os.getenv("POLYGON_API_KEY")
USE_POLYGON = bool(POLYGON_API_KEY)

if USE_POLYGON:
    print(f"✅ Polygon API key found - using Polygon as primary data source")
else:
    print(f"⚠️ No POLYGON_API_KEY found - using Yahoo Finance only")


def _to_dt(s: str) -> datetime:
    return datetime.strptime(s, "%Y-%m-%d")


def _unique_tickers(tickers: list[str]) -> list[str]:
    seen = set()
    out = []
    for t in tickers:
        t = str(t).strip().upper()
        if t and t not in seen:
            out.append(t)
            seen.add(t)
    return out


# ═══════════════════════════════════════════════════════════════════════════════
# POLYGON DATA FETCHING
# ═══════════════════════════════════════════════════════════════════════════════
def _fetch_polygon_daily(ticker: str, start_date: str, end_date: str, api_key: str) -> pd.DataFrame:
    """Fetch daily OHLCV from Polygon for a single ticker."""
    try:
        url = f"https://api.polygon.io/v2/aggs/ticker/{ticker}/range/1/day/{start_date}/{end_date}"
        resp = requests.get(
            url,
            params={"adjusted": "true", "sort": "asc", "limit": 5000, "apiKey": api_key},
            timeout=10,
        )
        resp.raise_for_status()
        results = resp.json().get("results") or []
        if not results:
            return pd.DataFrame()
        df = pd.DataFrame(results)
        if df.empty or not {"o", "h", "l", "c", "v", "t"}.issubset(df.columns):
            return pd.DataFrame()
        df["Date"] = pd.to_datetime(df["t"], unit="ms")
        df = df.rename(columns={"o": "Open", "h": "High", "l": "Low", "c": "Close", "v": "Volume"})
        df = df.set_index("Date")[["Open", "High", "Low", "Close", "Volume"]].dropna()
        return df
    except Exception:
        return pd.DataFrame()


def _download_polygon_batch(tickers: list[str], start_date: str, end_date: str, api_key: str, max_workers: int = 8) -> dict[str, pd.DataFrame]:
    """Download daily OHLCV for many tickers from Polygon."""
    results: dict[str, pd.DataFrame] = {}
    
    def worker(t: str):
        return t, _fetch_polygon_daily(t, start_date, end_date, api_key)
    
    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        futures = {ex.submit(worker, t): t for t in tickers}
        for fut in as_completed(futures):
            t, df = fut.result()
            results[t] = df
    return results


class StockTracker:
    def __init__(
        self,
        tickers: list[str],
        baseline_date: str,
        verbose: bool = True,
        auto_extend_if_short: bool = True,
        min_trading_days: int = 2,
        extra_backfill_days: int = 10,
    ):
        self.tickers = _unique_tickers(tickers)
        self.baseline_date = baseline_date
        self.verbose = verbose

        # robustness knobs (holiday/weekend edge cases)
        self.auto_extend_if_short = auto_extend_if_short
        self.min_trading_days = int(min_trading_days)
        self.extra_backfill_days = int(extra_backfill_days)

        # one-download store
        self.close: pd.DataFrame = pd.DataFrame()

        # derived
        self.baseline_prices: dict[str, float] = {}
        self.target_prices: dict[str, float] = {}

        # Global window timestamps (based on any ticker availability)
        self.window_start_ts: pd.Timestamp | None = None
        self.window_end_ts: pd.Timestamp | None = None

        # Per-ticker effective baseline date (first valid in window)
        self.per_ticker_baseline_ts: dict[str, pd.Timestamp] = {}

    def _log(self, msg: str):
        logging.info(msg)

    def _download_close_yfinance(self, start_dt: datetime, end_dt: datetime) -> pd.DataFrame:
        """Yahoo Finance download helper (fallback)."""
        data = yf.download(
            tickers=self.tickers,
            start=start_dt,
            end=end_dt,
            progress=False,
            auto_adjust=False,
            threads=True,
            group_by="column",
        )

        if data.empty:
            return pd.DataFrame()

        if isinstance(data.columns, pd.MultiIndex):
            close = data["Close"].copy()
        else:
            close = data[["Close"]].copy()
            close.columns = [self.tickers[0]]

        close = close.sort_index()
        return close

    def _download_close(self, start_dt: datetime, end_dt: datetime) -> pd.DataFrame:
        """
        Download Close prices. Uses Polygon if available, falls back to Yahoo Finance.
        """
        start_str = start_dt.strftime("%Y-%m-%d")
        end_str = end_dt.strftime("%Y-%m-%d")
        
        # Try Polygon first if available
        if USE_POLYGON:
            self._log(f"Using Polygon.io for {len(self.tickers)} tickers...")
            polygon_data = _download_polygon_batch(self.tickers, start_str, end_str, POLYGON_API_KEY)
            
            # Build Close DataFrame
            close_dfs = []
            success_count = 0
            failed_tickers = []
            
            for ticker in self.tickers:
                df = polygon_data.get(ticker, pd.DataFrame())
                if not df.empty and "Close" in df.columns:
                    close_dfs.append(df[["Close"]].rename(columns={"Close": ticker}))
                    success_count += 1
                else:
                    failed_tickers.append(ticker)
            
            self._log(f"Polygon: {success_count}/{len(self.tickers)} tickers succeeded")
            
            # Fall back to Yahoo Finance for failed tickers
            if failed_tickers and len(failed_tickers) < len(self.tickers):
                self._log(f"Falling back to Yahoo Finance for {len(failed_tickers)} tickers: {failed_tickers[:5]}...")
                try:
                    yf_data = yf.download(
                        tickers=failed_tickers,
                        start=start_dt,
                        end=end_dt,
                        progress=False,
                        auto_adjust=False,
                        threads=True,
                        group_by="column",
                    )
                    if not yf_data.empty:
                        if isinstance(yf_data.columns, pd.MultiIndex):
                            yf_close = yf_data["Close"].copy()
                        else:
                            yf_close = yf_data[["Close"]].copy()
                            yf_close.columns = [failed_tickers[0]]
                        for ticker in failed_tickers:
                            if ticker in yf_close.columns:
                                close_dfs.append(yf_close[[ticker]])
                except Exception:
                    pass
            
            # Combine all data
            if close_dfs:
                close = pd.concat(close_dfs, axis=1).sort_index()
                return close
            
            # If Polygon completely failed, fall back to Yahoo Finance
            self._log("Polygon failed completely, falling back to Yahoo Finance...")
        
        # Yahoo Finance fallback
        return self._download_close_yfinance(start_dt, end_dt)

    def download_close_once(self, end_date: str) -> pd.DataFrame:
        """
        One download for daily closes:
          baseline_date → end_date (+1 day)
        If too few trading days in the requested window (e.g., holiday/weekend),
        optionally extend backward by extra_backfill_days and re-download.
        """
        if not self.tickers:
            raise ValueError("STOCK_TICKERS is empty.")

        start_dt = _to_dt(self.baseline_date)
        end_dt = _to_dt(end_date) + timedelta(days=1)

        self._log(f"Downloading daily closes ONCE: {self.baseline_date} → {end_date} | tickers={len(self.tickers)}")
        t0 = time.time()

        close = self._download_close(start_dt, end_dt)

        if close.empty:
            self._log("No data returned from yfinance.")
            self.close = pd.DataFrame()
            return self.close

        # determine how many trading rows we have (rows where any ticker has data)
        valid_rows = close.dropna(how="all")
        if self.auto_extend_if_short and len(valid_rows) < self.min_trading_days:
            self._log(
                f"Only {len(valid_rows)} trading day(s) in range. "
                f"Extending backward by {self.extra_backfill_days} days and re-downloading."
            )
            start_dt2 = start_dt - timedelta(days=self.extra_backfill_days)
            close2 = self._download_close(start_dt2, end_dt)
            if not close2.empty:
                close = close2
            else:
                self._log("Extension download returned empty; keeping original window.")

        # forward fill helps missing values for some tickers on some days
        close = close.sort_index().ffill()

        dt_s = time.time() - t0
        self._log(f"Download complete in {dt_s:.2f}s | rows={len(close):,} | cols={len(close.columns)}")
        self.close = close
        return close

    def compute_baseline_and_target(self, end_date: str) -> tuple[dict[str, float], dict[str, float]]:
        """
        Per-ticker baseline/target within the requested window:
          - baseline = first valid close for ticker on/after baseline_date
          - target   = last valid close for ticker on/before end_date
        Also records:
          - global window start/end timestamps (based on any ticker availability)
          - per-ticker effective baseline timestamp
        """
        if self.close.empty:
            return {}, {}

        start_dt = _to_dt(self.baseline_date)
        end_dt = _to_dt(end_date)

        # Filter to the user's requested window (calendar), then drop all-NaN rows
        w = self.close.loc[(self.close.index >= start_dt) & (self.close.index <= end_dt)].copy()
        w = w.dropna(how="all")
        if w.empty:
            self._log("No valid trading rows inside the requested window after filtering.")
            return {}, {}

        self.window_start_ts = w.index[0]
        self.window_end_ts = w.index[-1]

        baseline_prices: dict[str, float] = {}
        target_prices: dict[str, float] = {}
        per_base_ts: dict[str, pd.Timestamp] = {}

        for t in self.tickers:
            if t not in w.columns:
                continue
            s = w[t].dropna()
            if s.empty:
                continue
            per_base_ts[t] = s.index[0]
            baseline_prices[t] = float(s.iloc[0])
            target_prices[t] = float(s.iloc[-1])

        # round for display
        self.baseline_prices = {k: round(v, 2) for k, v in baseline_prices.items()}
        self.target_prices = {k: round(v, 2) for k, v in target_prices.items()}
        self.per_ticker_baseline_ts = per_base_ts

        self._log(f"Window used (global): {self.window_start_ts.date()} → {self.window_end_ts.date()}")
        self._log(f"Valid tickers: {len(self.baseline_prices)}/{len(self.tickers)}")

        if self.verbose:
            self._log(f"Baseline prices: {self.baseline_prices}")
            self._log(f"Target prices:   {self.target_prices}")

        return self.baseline_prices, self.target_prices

    def make_returns_table(self) -> pd.DataFrame:
        """
        Performance table based on per-ticker baseline/target (alignment-safe).
        """
        rows = []
        for t in self.tickers:
            bp = self.baseline_prices.get(t)
            tp = self.target_prices.get(t)
            if bp is None or tp is None or bp == 0:
                continue
            change = tp - bp
            pct = (change / bp) * 100.0
            base_ts = self.per_ticker_baseline_ts.get(t)
            rows.append(
                {
                    "Symbol": t,
                    "Baseline": bp,
                    "Current": tp,
                    "Change": round(change, 2),
                    "Percent_Change": pct,
                    "Baseline_Date_Used": base_ts.date().isoformat() if isinstance(base_ts, pd.Timestamp) else None,
                }
            )

        df = pd.DataFrame(rows)
        if df.empty:
            return df

        df = df.sort_values("Percent_Change", ascending=False).reset_index(drop=True)
        return df

    def make_cumulative_returns_v2(self, end_date: str) -> tuple[pd.DataFrame, pd.Series, pd.DataFrame]:
        """
        Bulletproof v2 cumulative returns:
        - filters close to requested window [baseline_date, end_date]
        - per-ticker baseline alignment (anchor = first valid close per ticker within window)
        - trading-day counter (1..N) for x-axis
        Returns:
          df_cum: wide DataFrame, index=Date, cols=tickers, values=cum return %
          td_counter: Series indexed by Date, value=trading day number
          meta: DataFrame per ticker with effective baseline date + baseline price
        """
        if self.close.empty:
            return pd.DataFrame(), pd.Series(dtype=int), pd.DataFrame()

        start_dt = _to_dt(self.baseline_date)
        end_dt = _to_dt(end_date)

        w = self.close.loc[(self.close.index >= start_dt) & (self.close.index <= end_dt)].copy()
        w = w.dropna(how="all")
        if w.empty:
            return pd.DataFrame(), pd.Series(dtype=int), pd.DataFrame()

        # trading-day counter (only trading rows)
        td_counter = pd.Series(range(1, len(w.index) + 1), index=w.index, name="TradingDay")

        # per-ticker anchor price + baseline date
        anchors = {}
        base_dates = {}
        for t in self.tickers:
            if t not in w.columns:
                continue
            s = w[t].dropna()
            if s.empty:
                continue
            anchors[t] = float(s.iloc[0])
            base_dates[t] = s.index[0]

        if not anchors:
            return pd.DataFrame(), td_counter, pd.DataFrame()

        anchor_series = pd.Series(anchors)

        df_cum = (w.divide(anchor_series, axis="columns") - 1.0) * 100.0
        df_cum = df_cum.dropna(how="all")

        meta = pd.DataFrame(
            {
                "Ticker": list(anchors.keys()),
                "Baseline_Date_Used": [base_dates[t].date().isoformat() for t in anchors.keys()],
                "Baseline_Price_Used": [round(anchors[t], 2) for t in anchors.keys()],
            }
        ).set_index("Ticker")

        return df_cum, td_counter, meta

    def _get_bench_data(self, end_date: str) -> tuple[pd.DataFrame, pd.Series]:
        """Download and normalize benchmark close prices (S&P 500, Nasdaq 100)."""
        bench_symbols = ["^GSPC", "^NDX"]
        bench_label_map = {"^GSPC": "S&P 500", "^NDX": "Nasdaq 100"}
        bench_norm = pd.DataFrame()
        bench_returns = pd.Series(dtype=float)

        bench_raw = yf.download(bench_symbols, start=self.baseline_date, end=end_date)
        if not bench_raw.empty:
            # Reduce to a single price field (prefer Adj Close, else Close)
            if isinstance(bench_raw.columns, pd.MultiIndex):
                level0 = bench_raw.columns.get_level_values(0)
                if "Adj Close" in level0:
                    bench_raw = bench_raw["Adj Close"]
                elif "Close" in level0:
                    bench_raw = bench_raw["Close"]
                else:
                    bench_raw = bench_raw.xs(level0[0], level=0, axis=1)
            else:
                if "Adj Close" in bench_raw.columns:
                    bench_raw = bench_raw[["Adj Close"]]
                elif "Close" in bench_raw.columns:
                    bench_raw = bench_raw[["Close"]]
            bench_raw = bench_raw.ffill().bfill()
            if not bench_raw.empty:
                bench_norm = (bench_raw.divide(bench_raw.iloc[0]) - 1.0) * 100.0
                bench_norm = bench_norm.rename(columns=bench_label_map)
                bench_returns = ((bench_raw.iloc[-1] / bench_raw.iloc[0] - 1.0) * 100.0).rename(index=bench_label_map)

        return bench_norm, bench_returns

    def plot_bar_and_line_v2(self, df_perf: pd.DataFrame, end_date: str, bench_norm=None, bench_returns=None):
        """
        One figure, 2 rows:
          Row 1: total return bar chart
          Row 2: cumulative return paths with trading-day counter x-axis
        Hover shows actual date.
        """
        if df_perf.empty:
            self._log("No performance data to plot.")
            return

        df_cum, td_counter, meta = self.make_cumulative_returns_v2(end_date)

        if bench_norm is None or bench_returns is None:
            bench_norm, bench_returns = self._get_bench_data(end_date)

        if not df_cum.empty and isinstance(bench_norm, pd.DataFrame) and not bench_norm.empty:
            bench_norm = bench_norm.reindex(df_cum.index).ffill()

        bench_dash_cycle = ["dot", "dash"]
        bench_colors = ["#666", "#999"]

        # Titles
        g0 = self.window_start_ts.date().isoformat() if self.window_start_ts is not None else self.baseline_date
        g1 = self.window_end_ts.date().isoformat() if self.window_end_ts is not None else end_date

        title_top = f"📈 Total Return (per-ticker baseline): {g0} → {g1}"
        title_bottom = f"📉 Cumulative Return v2 (Start=0% per ticker) | X = Trading Day Counter"

        fig = make_subplots(
            rows=2,
            cols=1,
            shared_xaxes=False,
            vertical_spacing=0.12,
            subplot_titles=(title_top, title_bottom),
            row_heights=[0.45, 0.55],
        )

        # ----------------
        # Row 1: Bar chart
        # ----------------
        colors = np.where(df_perf["Percent_Change"] >= 0, "#00CC44", "#FF4444")
        fig.add_trace(
            go.Bar(
                x=df_perf["Symbol"],
                y=df_perf["Percent_Change"],
                marker_color=colors,
                text=[f"{x:.1f}%" for x in df_perf["Percent_Change"]],
                textposition="outside",
                hovertemplate=(
                    "<b>%{x}</b><br>"
                    "Baseline: $%{customdata[0]:.2f}<br>"
                    "Current: $%{customdata[1]:.2f}<br>"
                    "Return: %{y:.2f}%<br>"
                    "Baseline Date Used: %{customdata[2]}<extra></extra>"
                ),
                customdata=list(zip(df_perf["Baseline"], df_perf["Current"], df_perf["Baseline_Date_Used"])),
                showlegend=False,
            ),
            row=1,
            col=1,
        )

        avg_change = float(df_perf["Percent_Change"].mean())
        fig.add_hline(y=0, line_dash="dash", line_color="black", opacity=0.4, row=1, col=1)
        fig.add_hline(
            y=avg_change,
            line_dash="dash",
            line_color="blue",
            annotation_text=f"Avg: {avg_change:.2f}%",
            annotation_position="bottom right",
            row=1,
            col=1,
        )

        # ----------------------------
        # Row 2: Cum return (TradingDay)
        # ----------------------------
        if not df_cum.empty:
            # Make hover-friendly mapping: Date -> string
            date_str = pd.Series(df_cum.index.strftime("%Y-%m-%d"), index=df_cum.index)

            # Show markers if very short series
            mode = "lines"
            if len(df_cum.index) < 3:
                mode = "lines+markers"

            # plot each ticker that exists
            for t in self.tickers:
                if t not in df_cum.columns:
                    continue
                s = df_cum[t].dropna()
                if s.empty:
                    continue

                x_td = td_counter.loc[s.index].values
                y = s.values
                hover_dates = date_str.loc[s.index].values

                # per-ticker baseline info
                if t in meta.index:
                    bdate = meta.loc[t, "Baseline_Date_Used"]
                    bpx = meta.loc[t, "Baseline_Price_Used"]
                else:
                    bdate, bpx = "N/A", np.nan

                fig.add_trace(
                    go.Scatter(
                        x=x_td,
                        y=y,
                        mode=mode,
                        name=t,
                        customdata=np.column_stack([hover_dates]),
                        hovertemplate=(
                            f"<b>{t}</b><br>"
                            "Trading Day: %{x}<br>"
                            "Date: %{customdata[0]}<br>"
                            "Return: %{y:.2f}%<br>"
                            f"Baseline: {bdate} @ {bpx}<extra></extra>"
                        ),
                    ),
                    row=2,
                    col=1,
                )

            avg_path = df_cum.mean(axis=1, skipna=True)
            if not avg_path.empty:
                fig.add_trace(
                    go.Scatter(
                        x=td_counter.loc[avg_path.index].values,
                        y=avg_path.values,
                        mode="lines",
                        name="Portfolio avg (daily)",
                        line=dict(color="purple", width=3, dash="dashdot"),
                        hovertemplate="Trading Day: %{x}<br>Avg return: %{y:.2f}%<extra></extra>",
                    ),
                    row=2,
                    col=1,
                )

            if not bench_norm.empty:
                for i, col in enumerate(bench_norm.columns):
                    s = bench_norm[col].dropna()
                    if s.empty:
                        continue
                    x_td = td_counter.loc[s.index].values
                    hover_dates = date_str.loc[s.index].values
                    display_name = f"{col} benchmark" if "benchmark" not in str(col).lower() else str(col)
                    fig.add_trace(
                        go.Scatter(
                            x=x_td,
                            y=s.values,
                            mode="lines",
                            name=display_name,
                            line=dict(
                                color=bench_colors[i % len(bench_colors)],
                                dash=bench_dash_cycle[i % len(bench_dash_cycle)],
                                width=2,
                            ),
                            customdata=np.column_stack([hover_dates]),
                            hovertemplate=(
                                f"<b>{display_name}</b><br>"
                                "Trading Day: %{x}<br>"
                                "Date: %{customdata[0]}<br>"
                                "Return: %{y:.2f}%<extra></extra>"
                            ),
                        ),
                        row=2,
                        col=1,
                    )

            fig.add_hline(y=0, line_dash="dash", line_color="black", opacity=0.4, row=2, col=1)

            # nicer x ticks
            max_td = int(td_counter.max()) if not td_counter.empty else 1
            if max_td <= 15:
                fig.update_xaxes(dtick=1, row=2, col=1)
            elif max_td <= 60:
                fig.update_xaxes(dtick=5, row=2, col=1)
            else:
                fig.update_xaxes(dtick=10, row=2, col=1)

        fig.update_layout(
            height=920,
            width=1120,
            template="plotly_white",
            hovermode="closest",
            title={"text": f"Stock Tracker (One Download, Bulletproof v2) | End: {end_date}", "x": 0.5},
            legend_title_text="Tickers",
        )

        fig.update_xaxes(tickangle=45, row=1, col=1)
        fig.update_yaxes(title_text="Total Return (%)", row=1, col=1)

        fig.update_xaxes(title_text="Trading Day Counter (1 = first trading day in window)", row=2, col=1)
        fig.update_yaxes(title_text="Cumulative Return (%)", row=2, col=1)

        fig.show()


def run_trade_return(STOCK_TICKERS, BASELINE_DATE, future_date_str, group_sets: dict[str, list[str]] | None = None, verbose: bool = True):
    tracker = StockTracker(
        STOCK_TICKERS,
        BASELINE_DATE,
        verbose=verbose,
        auto_extend_if_short=True,
        min_trading_days=2,
        extra_backfill_days=10,
    )

    tracker.download_close_once(future_date_str)
    tracker.compute_baseline_and_target(future_date_str)

    df_perf = tracker.make_returns_table()
    if df_perf.empty:
        logging.info("No valid tickers with data. Nothing to report.")
        return None

    best = df_perf.iloc[0]
    worst = df_perf.iloc[-1]
    avg = float(df_perf["Percent_Change"].mean())

    bench_norm, bench_returns = tracker._get_bench_data(future_date_str)
    bench_summary = (
        "Bench=" + " | ".join([f"{k}={v:+.2f}%" for k, v in bench_returns.items()])
        if not bench_returns.empty
        else "Bench=N/A"
    )

    batch_summary = ""
    if group_sets:
        group_lookup = {
            "weekly_top5": group_sets.get("weekly", []),
            "pro30_tickers": group_sets.get("pro30", []),
            "movers_tickers": group_sets.get("movers", []),
        }
        batch_chunks = []
        for label, tickers in group_lookup.items():
            if not tickers:
                continue
            tickers_norm = [str(t).strip().upper() for t in tickers if str(t).strip()]
            sub = df_perf[df_perf["Symbol"].isin(tickers_norm)]
            if sub.empty:
                continue
            batch_chunks.append(f"{label}={sub['Percent_Change'].mean():+.2f}%")
        if batch_chunks:
            batch_summary = "Batches=" + " | ".join(batch_chunks)

    summary_msg = (
        f"Summary | Best={best['Symbol']} ({best['Percent_Change']:+.2f}%) "
        f"Worst={worst['Symbol']} ({worst['Percent_Change']:+.2f}%) "
        f"Avg={avg:+.2f}%"
    )

    tail_parts = [x for x in [batch_summary, bench_summary] if x]
    if tail_parts:
        summary_msg += "\n" + " | ".join(tail_parts)

    logging.info(summary_msg)

    tracker.plot_bar_and_line_v2(df_perf, future_date_str, bench_norm=bench_norm, bench_returns=bench_returns)
    return df_perf



✅ Polygon API key found - using Polygon as primary data source


In [2]:
import json
from pathlib import Path

def _dedup_keep_order(xs):
    out, seen = [], set()
    for x in xs:
        t = str(x).strip().upper()
        if t and t not in seen:
            out.append(t)
            seen.add(t)
    return out

def load_components_for_date(date_str: str, root: str = "outputs"):
    run_dir = Path(root) / date_str

    weekly = []   # list of dicts: {"ticker":..., "name":...}
    pro30 = []    # list of tickers
    movers = []   # list of tickers

    # Weekly Top 5 (preferred source)
    p_top5 = run_dir / f"weekly_scanner_top5_{date_str}.json"
    if p_top5.exists():
        obj = json.loads(p_top5.read_text())
        for x in obj.get("top5", []):
            if isinstance(x, dict) and x.get("ticker"):
                weekly.append({"ticker": x["ticker"], "name": x.get("name", "")})
    else:
        # fallback: hybrid_analysis embeds weekly_top5
        p_hybrid = run_dir / f"hybrid_analysis_{date_str}.json"
        if p_hybrid.exists():
            h = json.loads(p_hybrid.read_text())
            for x in h.get("weekly_top5", []):
                if isinstance(x, dict) and x.get("ticker"):
                    weekly.append({"ticker": x["ticker"], "name": x.get("name", "")})

    # Pro30 candidates (combined)
    p_pro30 = run_dir / f"30d_momentum_candidates_{date_str}.csv"
    if p_pro30.exists():
        df = pd.read_csv(p_pro30)
        if "Ticker" in df.columns:
            pro30 = df["Ticker"].dropna().tolist()
    else:
        # fallback: breakout/reversal files
        for fn in [f"30d_breakout_candidates_{date_str}.csv", f"30d_reversal_candidates_{date_str}.csv"]:
            p = run_dir / fn
            if p.exists():
                df = pd.read_csv(p)
                if "Ticker" in df.columns:
                    pro30 += df["Ticker"].dropna().tolist()

    # Movers (only stored in hybrid_analysis)
    p_hybrid = run_dir / f"hybrid_analysis_{date_str}.json"
    if p_hybrid.exists():
        h = json.loads(p_hybrid.read_text())
        movers = h.get("movers_tickers", []) or []

    weekly_tickers = _dedup_keep_order([w["ticker"] for w in weekly])
    pro30_tickers = _dedup_keep_order(pro30)
    movers_tickers = _dedup_keep_order(movers)

    # map ticker -> name (only guaranteed for weekly)
    name_map = {w["ticker"].strip().upper(): (w.get("name", "") or "") for w in weekly if w.get("ticker")}

    return {
        "weekly": weekly_tickers,
        "pro30": pro30_tickers,
        "movers": movers_tickers,
        "name_map": name_map,
    }

def format_pretty_tracker_block(date_str: str, root: str = "outputs") -> str:
    comp = load_components_for_date(date_str, root=root)
    weekly, pro30, movers = comp["weekly"], comp["pro30"], comp["movers"]

    lines = []
    lines.append(f"# Universe injected from outputs ({date_str})")
    lines.append(f"# - weekly_top5: {', '.join(weekly) if weekly else '(none)'}")
    lines.append(f"# - pro30_tickers: {', '.join(pro30) if pro30 else '(none)'}")
    lines.append(f"# - movers_tickers: {', '.join(movers) if movers else '(none)'}")
    return "\n".join(lines)


In [3]:
future_date_str = datetime.now().strftime("%Y-%m-%d")

In [4]:
BASELINE_DATE = '2026-01-01' 

# ChatGPT
STOCK_TICKERS = [
    'BMNR',    
    'MSTR',  
    'TSLA',  
    'CRCL',
    'IREN',  
    'SOFI',
    'OSCR',
    'BABA',
    'JD',
    'NVDA',
    'AMD',
    'GOOG',
    'AAPL',
    'UPXI',
]

result = run_trade_return(STOCK_TICKERS, BASELINE_DATE, future_date_str)

2026-01-17 17:15:03,987 | INFO | Downloading daily closes ONCE: 2026-01-01 → 2026-01-17 | tickers=14
2026-01-17 17:15:03,988 | INFO | Using Polygon.io for 14 tickers...
2026-01-17 17:15:05,702 | INFO | Polygon: 14/14 tickers succeeded
2026-01-17 17:15:05,706 | INFO | Download complete in 1.72s | rows=11 | cols=14
2026-01-17 17:15:05,708 | INFO | Window used (global): 2026-01-02 → 2026-01-16
2026-01-17 17:15:05,708 | INFO | Valid tickers: 14/14
2026-01-17 17:15:05,709 | INFO | Baseline prices: {'BMNR': 31.19, 'MSTR': 157.16, 'TSLA': 438.07, 'CRCL': 83.47, 'IREN': 42.7, 'SOFI': 27.46, 'OSCR': 14.97, 'BABA': 155.74, 'JD': 29.53, 'NVDA': 188.85, 'AMD': 223.47, 'GOOG': 315.32, 'AAPL': 271.01, 'UPXI': 1.91}
2026-01-17 17:15:05,709 | INFO | Target prices:   {'BMNR': 31.16, 'MSTR': 173.71, 'TSLA': 437.5, 'CRCL': 78.61, 'IREN': 57.82, 'SOFI': 26.13, 'OSCR': 16.55, 'BABA': 165.4, 'JD': 28.9, 'NVDA': 186.23, 'AMD': 231.83, 'GOOG': 330.34, 'AAPL': 255.53, 'UPXI': 2.31}


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  2 of 2 completed
2026-01-17 17:15:09,475 | INFO | Summary | Best=IREN (+35.41%) Worst=CRCL (-5.82%) Avg=+5.14%
Bench=S&P 500=+1.19% | Nasdaq 100=+1.28%


In [5]:
BASELINE_DATE = '2026-01-01' 

# ChatGPT
STOCK_TICKERS = [
    'SATS', 'MU', 'SPHR', 'LHX', 'LUNR', # weekly_top5
    'PATH', # pro30_tickers, # pro version
]

# optional: de-dup safeguard (keeps order)
STOCK_TICKERS = list(dict.fromkeys(STOCK_TICKERS))

result = run_trade_return(STOCK_TICKERS, BASELINE_DATE, future_date_str)


2026-01-17 17:15:09,859 | INFO | Downloading daily closes ONCE: 2026-01-01 → 2026-01-17 | tickers=6
2026-01-17 17:15:09,859 | INFO | Using Polygon.io for 6 tickers...
2026-01-17 17:15:10,645 | INFO | Polygon: 6/6 tickers succeeded
2026-01-17 17:15:10,646 | INFO | Download complete in 0.79s | rows=11 | cols=6
2026-01-17 17:15:10,648 | INFO | Window used (global): 2026-01-02 → 2026-01-16
2026-01-17 17:15:10,648 | INFO | Valid tickers: 6/6
2026-01-17 17:15:10,648 | INFO | Baseline prices: {'SATS': 112.18, 'MU': 315.42, 'SPHR': 94.3, 'LHX': 304.48, 'LUNR': 17.88, 'PATH': 15.88}
2026-01-17 17:15:10,649 | INFO | Target prices:   {'SATS': 123.27, 'MU': 362.75, 'SPHR': 96.36, 'LHX': 346.46, 'LUNR': 21.58, 'PATH': 14.34}
[*********************100%***********************]  2 of 2 completed
2026-01-17 17:15:10,668 | INFO | Summary | Best=LUNR (+20.69%) Worst=PATH (-9.70%) Avg=+8.64%
Bench=S&P 500=+1.19% | Nasdaq 100=+1.28%


In [6]:
BASELINE_DATE = "2026-01-02"
print(format_pretty_tracker_block(BASELINE_DATE))
comp = load_components_for_date(BASELINE_DATE)
STOCK_TICKERS = _dedup_keep_order(comp["weekly"] + comp["pro30"] + comp["movers"])
result = run_trade_return(STOCK_TICKERS, BASELINE_DATE, future_date_str, group_sets=comp)

2026-01-17 17:15:10,724 | INFO | Downloading daily closes ONCE: 2026-01-02 → 2026-01-17 | tickers=2
2026-01-17 17:15:10,724 | INFO | Using Polygon.io for 2 tickers...


# Universe injected from outputs (2026-01-02)
# - weekly_top5: (none)
# - pro30_tickers: PATH
# - movers_tickers: CVX


2026-01-17 17:15:11,628 | INFO | Polygon: 2/2 tickers succeeded
2026-01-17 17:15:11,631 | INFO | Download complete in 0.91s | rows=11 | cols=2
2026-01-17 17:15:11,633 | INFO | Window used (global): 2026-01-02 → 2026-01-16
2026-01-17 17:15:11,634 | INFO | Valid tickers: 2/2
2026-01-17 17:15:11,635 | INFO | Baseline prices: {'PATH': 15.88, 'CVX': 155.9}
2026-01-17 17:15:11,635 | INFO | Target prices:   {'PATH': 14.34, 'CVX': 166.26}
[*********************100%***********************]  2 of 2 completed
2026-01-17 17:15:16,631 | INFO | Summary | Best=CVX (+6.65%) Worst=PATH (-9.70%) Avg=-1.53%
Batches=pro30_tickers=-9.70% | movers_tickers=+6.65% | Bench=S&P 500=+1.19% | Nasdaq 100=+1.28%


In [7]:
BASELINE_DATE = "2026-01-03"
print(format_pretty_tracker_block(BASELINE_DATE))
comp = load_components_for_date(BASELINE_DATE)
STOCK_TICKERS = _dedup_keep_order(comp["weekly"] + comp["pro30"] + comp["movers"])
result = run_trade_return(STOCK_TICKERS, BASELINE_DATE, future_date_str, group_sets=comp)

2026-01-17 17:15:16,679 | INFO | Downloading daily closes ONCE: 2026-01-03 → 2026-01-17 | tickers=6
2026-01-17 17:15:16,679 | INFO | Using Polygon.io for 6 tickers...


# Universe injected from outputs (2026-01-03)
# - weekly_top5: DBRG, RKLB, SATS, HII, HWM
# - pro30_tickers: (none)
# - movers_tickers: CVX


2026-01-17 17:15:17,489 | INFO | Polygon: 6/6 tickers succeeded
2026-01-17 17:15:17,491 | INFO | Download complete in 0.81s | rows=10 | cols=6
2026-01-17 17:15:17,493 | INFO | Window used (global): 2026-01-05 → 2026-01-16
2026-01-17 17:15:17,493 | INFO | Valid tickers: 6/6
2026-01-17 17:15:17,493 | INFO | Baseline prices: {'DBRG': 15.31, 'RKLB': 78.14, 'SATS': 110.67, 'HII': 363.48, 'HWM': 212.92, 'CVX': 163.85}
2026-01-17 17:15:17,494 | INFO | Target prices:   {'DBRG': 15.35, 'RKLB': 96.3, 'SATS': 123.27, 'HII': 425.9, 'HWM': 224.89, 'CVX': 166.26}
[*********************100%***********************]  2 of 2 completed
2026-01-17 17:15:19,396 | INFO | Summary | Best=RKLB (+23.24%) Worst=DBRG (+0.26%) Avg=+9.86%
Batches=weekly_top5=+11.54% | movers_tickers=+1.47% | Bench=S&P 500=+0.55% | Nasdaq 100=+0.50%


In [8]:
BASELINE_DATE = "2026-01-05"
print(format_pretty_tracker_block(BASELINE_DATE))
comp = load_components_for_date(BASELINE_DATE)
STOCK_TICKERS = _dedup_keep_order(comp["weekly"] + comp["pro30"] + comp["movers"])
result = run_trade_return(STOCK_TICKERS, BASELINE_DATE, future_date_str, group_sets=comp)

2026-01-17 17:15:19,445 | INFO | Downloading daily closes ONCE: 2026-01-05 → 2026-01-17 | tickers=5
2026-01-17 17:15:19,446 | INFO | Using Polygon.io for 5 tickers...


# Universe injected from outputs (2026-01-05)
# - weekly_top5: HAL, AXSM, DBRG, DOCN, EL
# - pro30_tickers: (none)
# - movers_tickers: (none)


2026-01-17 17:15:20,269 | INFO | Polygon: 5/5 tickers succeeded
2026-01-17 17:15:20,271 | INFO | Download complete in 0.83s | rows=10 | cols=5
2026-01-17 17:15:20,273 | INFO | Window used (global): 2026-01-05 → 2026-01-16
2026-01-17 17:15:20,273 | INFO | Valid tickers: 5/5
2026-01-17 17:15:20,274 | INFO | Baseline prices: {'HAL': 31.92, 'AXSM': 171.49, 'DBRG': 15.31, 'DOCN': 50.61, 'EL': 108.16}
2026-01-17 17:15:20,274 | INFO | Target prices:   {'HAL': 32.57, 'AXSM': 177.27, 'DBRG': 15.35, 'DOCN': 52.76, 'EL': 115.05}
[*********************100%***********************]  2 of 2 completed
2026-01-17 17:15:23,927 | INFO | Summary | Best=EL (+6.37%) Worst=DBRG (+0.26%) Avg=+3.26%
Batches=weekly_top5=+3.26% | Bench=S&P 500=+0.55% | Nasdaq 100=+0.50%


In [9]:
BASELINE_DATE = "2026-01-06"
print(format_pretty_tracker_block(BASELINE_DATE))
comp = load_components_for_date(BASELINE_DATE)
STOCK_TICKERS = _dedup_keep_order(comp["weekly"] + comp["pro30"] + comp["movers"])
result = run_trade_return(STOCK_TICKERS, BASELINE_DATE, future_date_str, group_sets=comp)

2026-01-17 17:15:23,985 | INFO | Downloading daily closes ONCE: 2026-01-06 → 2026-01-17 | tickers=8
2026-01-17 17:15:23,985 | INFO | Using Polygon.io for 8 tickers...


# Universe injected from outputs (2026-01-06)
# - weekly_top5: LITE, BK, DAL, INCY, PNC
# - pro30_tickers: SNDK, WDC, CENX
# - movers_tickers: (none)


2026-01-17 17:15:25,182 | INFO | Polygon: 8/8 tickers succeeded
2026-01-17 17:15:25,184 | INFO | Download complete in 1.20s | rows=9 | cols=8
2026-01-17 17:15:25,187 | INFO | Window used (global): 2026-01-06 → 2026-01-16
2026-01-17 17:15:25,187 | INFO | Valid tickers: 8/8
2026-01-17 17:15:25,188 | INFO | Baseline prices: {'LITE': 397.42, 'BK': 121.31, 'DAL': 72.22, 'INCY': 106.66, 'PNC': 218.04, 'SNDK': 349.63, 'WDC': 219.38, 'CENX': 42.23}
2026-01-17 17:15:25,188 | INFO | Target prices:   {'LITE': 324.25, 'BK': 121.33, 'DAL': 70.43, 'INCY': 106.21, 'PNC': 223.18, 'SNDK': 413.62, 'WDC': 221.51, 'CENX': 47.57}
[*********************100%***********************]  2 of 2 completed
2026-01-17 17:15:27,358 | INFO | Summary | Best=SNDK (+18.30%) Worst=LITE (-18.41%) Avg=+1.62%
Batches=weekly_top5=-3.79% | pro30_tickers=+10.64% | Bench=S&P 500=-0.07% | Nasdaq 100=-0.43%


In [10]:
BASELINE_DATE = "2026-01-07"
print(format_pretty_tracker_block(BASELINE_DATE))
comp = load_components_for_date(BASELINE_DATE)
STOCK_TICKERS = _dedup_keep_order(comp["weekly"] + comp["pro30"] + comp["movers"])
result = run_trade_return(STOCK_TICKERS, BASELINE_DATE, future_date_str, group_sets=comp)

2026-01-17 17:15:27,413 | INFO | Downloading daily closes ONCE: 2026-01-07 → 2026-01-17 | tickers=7
2026-01-17 17:15:27,413 | INFO | Using Polygon.io for 7 tickers...


# Universe injected from outputs (2026-01-07)
# - weekly_top5: PSX, ABBV, AEIS, GH, VSEC
# - pro30_tickers: MOD, INTC
# - movers_tickers: (none)


2026-01-17 17:15:28,243 | INFO | Polygon: 7/7 tickers succeeded
2026-01-17 17:15:28,245 | INFO | Download complete in 0.83s | rows=8 | cols=7
2026-01-17 17:15:28,247 | INFO | Window used (global): 2026-01-07 → 2026-01-16
2026-01-17 17:15:28,247 | INFO | Valid tickers: 7/7
2026-01-17 17:15:28,248 | INFO | Baseline prices: {'PSX': 137.88, 'ABBV': 233.42, 'AEIS': 224.4, 'GH': 109.93, 'VSEC': 195.27, 'MOD': 119.68, 'INTC': 42.63}
2026-01-17 17:15:28,248 | INFO | Target prices:   {'PSX': 138.28, 'ABBV': 214.35, 'AEIS': 253.86, 'GH': 112.14, 'VSEC': 214.18, 'MOD': 136.88, 'INTC': 46.96}
[*********************100%***********************]  2 of 2 completed
2026-01-17 17:15:30,151 | INFO | Summary | Best=MOD (+14.37%) Worst=ABBV (-8.17%) Avg=+5.92%
Batches=weekly_top5=+3.39% | pro30_tickers=+12.26% | Bench=S&P 500=+0.28% | Nasdaq 100=-0.49%


In [11]:
BASELINE_DATE = "2026-01-08"
print(format_pretty_tracker_block(BASELINE_DATE))
comp = load_components_for_date(BASELINE_DATE)
STOCK_TICKERS = _dedup_keep_order(comp["weekly"] + comp["pro30"] + comp["movers"])
result = run_trade_return(STOCK_TICKERS, BASELINE_DATE, future_date_str, group_sets=comp)

2026-01-17 17:15:30,203 | INFO | Downloading daily closes ONCE: 2026-01-08 → 2026-01-17 | tickers=6
2026-01-17 17:15:30,203 | INFO | Using Polygon.io for 6 tickers...


# Universe injected from outputs (2026-01-08)
# - weekly_top5: NOC, F, BBIO, COF, BFH
# - pro30_tickers: HOUS
# - movers_tickers: (none)


2026-01-17 17:15:31,041 | INFO | Polygon: 6/6 tickers succeeded
2026-01-17 17:15:31,046 | INFO | Download complete in 0.84s | rows=7 | cols=6
2026-01-17 17:15:31,048 | INFO | Window used (global): 2026-01-08 → 2026-01-16
2026-01-17 17:15:31,048 | INFO | Valid tickers: 6/6
2026-01-17 17:15:31,048 | INFO | Baseline prices: {'NOC': 590.79, 'F': 14.4, 'BBIO': 73.82, 'COF': 255.68, 'BFH': 81.25, 'HOUS': 17.64}
2026-01-17 17:15:31,049 | INFO | Target prices:   {'NOC': 666.9, 'F': 13.6, 'BBIO': 76.26, 'COF': 239.14, 'BFH': 73.73, 'HOUS': 17.64}
[*********************100%***********************]  2 of 2 completed
2026-01-17 17:15:34,084 | INFO | Summary | Best=NOC (+12.88%) Worst=BFH (-9.26%) Avg=-0.85%
Batches=weekly_top5=-1.02% | pro30_tickers=+0.00% | Bench=S&P 500=+0.27% | Nasdaq 100=+0.09%


In [12]:
BASELINE_DATE = "2026-01-09"
print(format_pretty_tracker_block(BASELINE_DATE))
comp = load_components_for_date(BASELINE_DATE)
STOCK_TICKERS = _dedup_keep_order(comp["weekly"] + comp["pro30"] + comp["movers"])
result = run_trade_return(STOCK_TICKERS, BASELINE_DATE, future_date_str, group_sets=comp)

2026-01-17 17:15:34,131 | INFO | Downloading daily closes ONCE: 2026-01-09 → 2026-01-17 | tickers=13
2026-01-17 17:15:34,131 | INFO | Using Polygon.io for 13 tickers...


# Universe injected from outputs (2026-01-09)
# - weekly_top5: BK, DAL, BBIO, DG, CBOE
# - pro30_tickers: LQDA, CGON, GLUE, APLD, HOUS, KTOS, COMP, INTC
# - movers_tickers: (none)


2026-01-17 17:15:35,796 | INFO | Polygon: 12/13 tickers succeeded
2026-01-17 17:15:35,796 | INFO | Falling back to Yahoo Finance for 1 tickers: ['HOUS']...
2026-01-17 17:15:37,125 | INFO | Download complete in 2.99s | rows=7 | cols=13
2026-01-17 17:15:37,128 | INFO | Window used (global): 2026-01-09 → 2026-01-16
2026-01-17 17:15:37,129 | INFO | Valid tickers: 13/13
2026-01-17 17:15:37,129 | INFO | Baseline prices: {'BK': 119.04, 'DAL': 72.31, 'BBIO': 77.25, 'DG': 142.74, 'CBOE': 260.95, 'LQDA': 35.86, 'CGON': 54.2, 'GLUE': 24.28, 'APLD': 37.68, 'HOUS': 17.64, 'KTOS': 113.7, 'COMP': 12.84, 'INTC': 45.55}
2026-01-17 17:15:37,129 | INFO | Target prices:   {'BK': 121.33, 'DAL': 70.43, 'BBIO': 76.26, 'DG': 148.74, 'CBOE': 273.55, 'LQDA': 37.93, 'CGON': 54.73, 'GLUE': 23.71, 'APLD': 37.4, 'HOUS': 17.64, 'KTOS': 130.72, 'COMP': 12.86, 'INTC': 46.96}
[*********************100%***********************]  2 of 2 completed
2026-01-17 17:15:39,219 | INFO | Summary | Best=KTOS (+14.97%) Worst=DAL (-2

In [13]:
BASELINE_DATE = "2026-01-12"
print(format_pretty_tracker_block(BASELINE_DATE))
comp = load_components_for_date(BASELINE_DATE)
STOCK_TICKERS = _dedup_keep_order(comp["weekly"] + comp["pro30"] + comp["movers"])
result = run_trade_return(STOCK_TICKERS, BASELINE_DATE, future_date_str, group_sets=comp)

2026-01-17 17:15:39,281 | INFO | Downloading daily closes ONCE: 2026-01-12 → 2026-01-17 | tickers=7
2026-01-17 17:15:39,282 | INFO | Using Polygon.io for 7 tickers...


# Universe injected from outputs (2026-01-12)
# - weekly_top5: GD, AKAM, EXPE, CADE, NOC
# - pro30_tickers: LQDA, SLNO
# - movers_tickers: (none)


2026-01-17 17:15:40,119 | INFO | Polygon: 7/7 tickers succeeded
2026-01-17 17:15:40,121 | INFO | Download complete in 0.84s | rows=5 | cols=7
2026-01-17 17:15:40,123 | INFO | Window used (global): 2026-01-12 → 2026-01-16
2026-01-17 17:15:40,123 | INFO | Valid tickers: 7/7
2026-01-17 17:15:40,124 | INFO | Baseline prices: {'GD': 360.94, 'AKAM': 91.27, 'EXPE': 291.53, 'CADE': 44.21, 'NOC': 629.32, 'LQDA': 36.6, 'SLNO': 41.54}
2026-01-17 17:15:40,124 | INFO | Target prices:   {'GD': 367.38, 'AKAM': 93.49, 'EXPE': 285.11, 'CADE': 44.5, 'NOC': 666.9, 'LQDA': 37.93, 'SLNO': 42.48}
[*********************100%***********************]  2 of 2 completed
2026-01-17 17:15:42,026 | INFO | Summary | Best=NOC (+5.97%) Worst=EXPE (-2.20%) Avg=+2.08%
Batches=weekly_top5=+1.73% | pro30_tickers=+2.95% | Bench=S&P 500=-0.53% | Nasdaq 100=-1.00%


In [14]:
BASELINE_DATE = "2026-01-13"
print(format_pretty_tracker_block(BASELINE_DATE))
comp = load_components_for_date(BASELINE_DATE)
STOCK_TICKERS = _dedup_keep_order(comp["weekly"] + comp["pro30"] + comp["movers"])
result = run_trade_return(STOCK_TICKERS, BASELINE_DATE, future_date_str, group_sets=comp)

2026-01-17 17:15:42,080 | INFO | Downloading daily closes ONCE: 2026-01-13 → 2026-01-17 | tickers=12
2026-01-17 17:15:42,080 | INFO | Using Polygon.io for 12 tickers...


# Universe injected from outputs (2026-01-13)
# - weekly_top5: SATS, OPCH, BBIO, FORM, RKLB
# - pro30_tickers: TVTX, TTMI, ATEC, HOUS, FIG, FORM, STNG, TGTX
# - movers_tickers: (none)


2026-01-17 17:15:43,648 | INFO | Polygon: 11/12 tickers succeeded
2026-01-17 17:15:43,649 | INFO | Falling back to Yahoo Finance for 1 tickers: ['HOUS']...
2026-01-17 17:15:44,884 | ERROR | 
1 Failed download:
2026-01-17 17:15:44,886 | ERROR | ['HOUS']: YFPricesMissingError('possibly delisted; no price data found  (1d 2026-01-13 00:00:00 -> 2026-01-18 00:00:00)')
2026-01-17 17:15:44,894 | INFO | Download complete in 2.81s | rows=4 | cols=11
2026-01-17 17:15:44,898 | INFO | Window used (global): 2026-01-13 → 2026-01-16
2026-01-17 17:15:44,899 | INFO | Valid tickers: 11/12
2026-01-17 17:15:44,900 | INFO | Baseline prices: {'SATS': 123.74, 'OPCH': 34.75, 'BBIO': 79.11, 'FORM': 72.55, 'RKLB': 86.58, 'TVTX': 29.11, 'TTMI': 93.24, 'ATEC': 16.45, 'FIG': 33.56, 'STNG': 60.09, 'TGTX': 27.87}
2026-01-17 17:15:44,900 | INFO | Target prices:   {'SATS': 123.27, 'OPCH': 36.03, 'BBIO': 76.26, 'FORM': 78.31, 'RKLB': 96.3, 'TVTX': 27.87, 'TTMI': 101.01, 'ATEC': 17.74, 'FIG': 29.57, 'STNG': 59.08, 'TGTX

In [15]:
BASELINE_DATE = "2026-01-14"
print(format_pretty_tracker_block(BASELINE_DATE))
comp = load_components_for_date(BASELINE_DATE)
STOCK_TICKERS = _dedup_keep_order(comp["weekly"] + comp["pro30"] + comp["movers"])
result = run_trade_return(STOCK_TICKERS, BASELINE_DATE, future_date_str, group_sets=comp)

2026-01-17 17:15:46,830 | INFO | Downloading daily closes ONCE: 2026-01-14 → 2026-01-17 | tickers=6
2026-01-17 17:15:46,830 | INFO | Using Polygon.io for 6 tickers...


# Universe injected from outputs (2026-01-14)
# - weekly_top5: AKAM, MRNA, INTC, RVTY, ON
# - pro30_tickers: APP
# - movers_tickers: (none)


2026-01-17 17:15:47,690 | INFO | Polygon: 6/6 tickers succeeded
2026-01-17 17:15:47,692 | INFO | Download complete in 0.86s | rows=3 | cols=6
2026-01-17 17:15:47,694 | INFO | Window used (global): 2026-01-14 → 2026-01-16
2026-01-17 17:15:47,694 | INFO | Valid tickers: 6/6
2026-01-17 17:15:47,694 | INFO | Baseline prices: {'AKAM': 90.65, 'MRNA': 40.58, 'INTC': 48.72, 'RVTY': 111.52, 'ON': 60.58, 'APP': 617.76}
2026-01-17 17:15:47,695 | INFO | Target prices:   {'AKAM': 93.49, 'MRNA': 41.83, 'INTC': 46.96, 'RVTY': 112.5, 'ON': 60.33, 'APP': 568.76}
[*********************100%***********************]  2 of 2 completed
2026-01-17 17:15:51,056 | INFO | Summary | Best=AKAM (+3.13%) Worst=APP (-7.93%) Avg=-0.81%
Batches=weekly_top5=+0.61% | pro30_tickers=-7.93% | Bench=S&P 500=+0.19% | Nasdaq 100=+0.25%


In [18]:
BASELINE_DATE = "2026-01-15"
print(format_pretty_tracker_block(BASELINE_DATE))
comp = load_components_for_date(BASELINE_DATE)
STOCK_TICKERS = _dedup_keep_order(comp["weekly"] + comp["pro30"] + comp["movers"])
result = run_trade_return(STOCK_TICKERS, BASELINE_DATE, future_date_str, group_sets=comp)

2026-01-18 13:12:30,228 | INFO | Downloading daily closes ONCE: 2026-01-15 → 2026-01-17 | tickers=17
2026-01-18 13:12:30,229 | INFO | Using Polygon.io for 17 tickers...


# Universe injected from outputs (2026-01-15)
# - weekly_top5: MS, CADE, DAL
# - pro30_tickers: OCUL, COHR, HOUS, SEI, ATEC, BRZE, DVN
# - movers_tickers: CRM, FIG, IRTC, PATH, STNG, TVTX, WGS


2026-01-18 13:12:32,697 | INFO | Polygon: 16/17 tickers succeeded
2026-01-18 13:12:32,697 | INFO | Falling back to Yahoo Finance for 1 tickers: ['HOUS']...
2026-01-18 13:12:35,090 | ERROR | 
1 Failed download:
2026-01-18 13:12:35,092 | ERROR | ['HOUS']: YFPricesMissingError('possibly delisted; no price data found  (1d 2026-01-15 00:00:00 -> 2026-01-18 00:00:00)')
2026-01-18 13:12:35,102 | INFO | Download complete in 4.87s | rows=2 | cols=16
2026-01-18 13:12:35,105 | INFO | Window used (global): 2026-01-15 → 2026-01-16
2026-01-18 13:12:35,105 | INFO | Valid tickers: 16/17
2026-01-18 13:12:35,106 | INFO | Baseline prices: {'MS': 191.23, 'CADE': 44.4, 'DAL': 71.34, 'OCUL': 11.28, 'COHR': 195.96, 'SEI': 55.4, 'ATEC': 17.46, 'BRZE': 25.48, 'DVN': 36.32, 'CRM': 233.53, 'FIG': 31.6, 'IRTC': 172.04, 'PATH': 14.79, 'STNG': 58.37, 'TVTX': 28.4, 'WGS': 109.28}
2026-01-18 13:12:35,107 | INFO | Target prices:   {'MS': 189.09, 'CADE': 44.5, 'DAL': 70.43, 'OCUL': 11.27, 'COHR': 191.04, 'SEI': 56.87, 

In [19]:
BASELINE_DATE = "2026-01-16"
print(format_pretty_tracker_block(BASELINE_DATE))
comp = load_components_for_date(BASELINE_DATE)
STOCK_TICKERS = _dedup_keep_order(comp["weekly"] + comp["pro30"] + comp["movers"])
result = run_trade_return(STOCK_TICKERS, BASELINE_DATE, future_date_str, group_sets=comp)

2026-01-18 19:31:09,477 | INFO | Downloading daily closes ONCE: 2026-01-16 → 2026-01-17 | tickers=28
2026-01-18 19:31:09,477 | INFO | Using Polygon.io for 28 tickers...


# Universe injected from outputs (2026-01-16)
# - weekly_top5: CLSK, NUE, MS
# - pro30_tickers: OCUL, COHR, HOUS, SEI, ATEC, BRZE
# - movers_tickers: AEIS, AFCG, APP, ATEC, CRCL, CRM, DAVE, FIG, GKOS, IRON, IRTC, LZ, PATH, PI, PXED, STNG, TVTX, UEC, VIA, WGS


2026-01-18 19:31:12,725 | INFO | Polygon: 27/28 tickers succeeded
2026-01-18 19:31:12,725 | INFO | Falling back to Yahoo Finance for 1 tickers: ['HOUS']...
2026-01-18 19:31:13,838 | ERROR | 
1 Failed download:
2026-01-18 19:31:13,838 | ERROR | ['HOUS']: YFPricesMissingError('possibly delisted; no price data found  (1d 2026-01-16 00:00:00 -> 2026-01-18 00:00:00)')
2026-01-18 19:31:13,848 | INFO | Only 1 trading day(s) in range. Extending backward by 10 days and re-downloading.
2026-01-18 19:31:13,849 | INFO | Using Polygon.io for 28 tickers...
2026-01-18 19:31:19,694 | INFO | Polygon: 28/28 tickers succeeded
2026-01-18 19:31:19,697 | INFO | Download complete in 10.22s | rows=9 | cols=28
2026-01-18 19:31:19,700 | INFO | Window used (global): 2026-01-16 → 2026-01-16
2026-01-18 19:31:19,700 | INFO | Valid tickers: 28/28
2026-01-18 19:31:19,701 | INFO | Baseline prices: {'CLSK': 13.37, 'NUE': 174.39, 'MS': 189.09, 'OCUL': 11.27, 'COHR': 191.04, 'HOUS': 17.64, 'SEI': 56.87, 'ATEC': 17.74, 'B